Progetto: **Classificazione di testi descrittivi per destinazione d'uso, inerenti ai beni sottoposti ad aste giudiziarie italiane.**

Studente: **Alessandro Monolo** | *10439147*

Relatore: Marco Brambilla

Referente aziendale: Simone Redaelli

Master: Data Science & Artificial Intelligence

Università: Politecnico di Milano

<hr style="border:1px solid black">

## Gestione dei valori ambigui nella colonna "Numero Vani", drop dei dati duplicati e fill delle null values nella colonna "Superficie" & "Numero Vani"

- **1.** Controllo la presenza di valori pari a 0 nella colonna Numero Vani & Superficie.
    Laddove ci siano valori pari a 0, approssimare per eccesso sostituendo i valori trovati con 0.5, ovvero il valore minimo per un vano;
    

- **2.** Controllo la presenza di valori duplicati all'interno del dataframe, selezionando solo i valori univoci per ogni colonna;


- **3.** Mappo le Null values nella colonna Superficie, estraendo con una Regex i valori dei metri quadrati dalla descrizione del bene;


- **4.** Dai valori estratti dalla regex per il campo Superficie, ora mappo le null values nel campo Numero Vani con i metri quadrati estratti;


- **5.** Conclusioni;


- **6.** Esporto il data frame in un file CSV;

<hr style="border:1px solid black">

**Importo le librerie che mi servono:**

In [1]:
import pandas as pd
import numpy as np
import re

#### Set pandas options:

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 1000
pd.options.display.float_format = '{:.2f}'.format

**Importo file CSV**

In [3]:
df = pd.read_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\3_Dataset_Cleaning_2\\Dataset_Cleaning_2.csv",
                 dtype={'Numero_Lotto' : 'Int64'})

### 1 - Controllo la presenza di valori pari a 0 nelle "Numero Vani" e "Superficie":


- La colonna **Numero_Vani** presenta **1341** valori **uguali a zero**.
- Questi valori sono stati generati dalla divisione / 20 della relativa superfice e dalla successiva approssimazione.
- Questa colonna non dovrebbe avere 0, quindi saranno approssimati per 0.5, come valore minimo di vani.
- La colonna superficie **non presenta valori pari a 0**.

In [4]:
# Conto il numero di 0.00 presenti prima della conversione degli 0:
print("Numero di 0 presenti nella colonna Numero Vani prima: ", '\033[1m','\033[15;31;43m', 
      df[df['Numero_Vani'] == 0].shape[0], '\033[0m')
# Conto il numero di 0.5 presenti prima della conversione degli 0:
print("Numero di 0.5 presenti nella colonna Numero Vani prima: ", '\033[1m','\033[15;31;43m', 
      df[df['Numero_Vani'] == 0.5].shape[0], '\033[0m')

# Converto gli 0 in 0.5 come valore minimo ammesso nella colonna Numero_Vani:
df['Numero_Vani'] = df['Numero_Vani'].apply(lambda x: 0.5 if x == 0.00 else x)

# Conto il numero di 0.00 presenti dopo la conversione degli 0:
print("Numero di 0 presenti nella colonna Numero Vani dopo: ", '\033[1m','\033[15;31;43m', 
      df[df['Numero_Vani'] == 0].shape[0], '\033[0m')
# Conto il numero di 0.5 presenti dopo la conversione degli 0:
print("Numero di 0.5 presenti nella colonna Numero Vani dopo: ", '\033[1m','\033[15;31;43m', 
      df[df['Numero_Vani'] == 0.5].shape[0], '\033[0m')

# Conto il numero di 0.00 presenti prima della conversione degli 0:
print("Numero di 0 presenti nella colonna Superficie: ", '\033[1m','\033[15;31;43m', 
      df[df['Superficie'] == 0].shape[0], '\033[0m')

Numero di 0 presenti nella colonna Numero Vani prima:    1341 
Numero di 0.5 presenti nella colonna Numero Vani prima:    14140 
Numero di 0 presenti nella colonna Numero Vani dopo:    0 
Numero di 0.5 presenti nella colonna Numero Vani dopo:    15481 
Numero di 0 presenti nella colonna Superficie:    0 


### 2 - Controllo la presenza di valori multipli nel data frame:


- Verifico la presenza di eventuali duplicates;
- Nel caso siano presenti li elimino dal data frame.

In [5]:
print("Numero beni prima di trovare i duplicati: ",
      '\033[1m','\033[15;31;43m', df.shape[0], '\033[0m')

# Indico le colonne utili per trovare in esse duplicat nel data frame:
colonne_df_2 = ['Destinazione_Uso',
              'Categoria_Catastale',
              'Tribunale',
              'Provincia',
              'Comune',
              'Numero_Lotto',
              'Numero_Vani',
              'Superficie',
              'Catasto_Fabbricati',
              'Descrizione_Bene',
              'N_Caratteri'] 

# Elimino i valori multipli presenti nel data frame secondo le colonne precisate sopra:
df = df.drop_duplicates(subset = colonne_df_2)

# Scrivo considerazioni:
print("Numero beni dopo aver trovato ed eliminato i doppioni: ",
      '\033[1m','\033[15;31;43m', df.shape[0], '\033[0m')
# Conclusioni sui duplicati:
print("Numero totale di beni eliminati dopo aver trovato i duplicati: ",
      '\033[1m','\033[15;31;43m', 221812 - 194553, '\033[0m')

Numero beni prima di trovare i duplicati:    221812 
Numero beni dopo aver trovato ed eliminato i doppioni:    194553 
Numero totale di beni eliminati dopo aver trovato i duplicati:    27259 


### 3 - Estraggo i valori dei mq dalla descrizione del bene per mappare il campo Superficie quando è mancante:

- Estraggo valori numerici presenti in prossimità delle scritte *mq, MQ, Mq, M2, m2 ,metri quadrati* neli testi della colonna descrizione;
- Mappo le Null Values presenti nella colonna superficie con i valori estratt dalla regex;

In [6]:
print("Prima di bonificare le null values, ci sono ora", '\033[1m','\033[15;31;43m', df['Superficie'].isnull().sum(),
      '\033[0m', "valori mancanti nella colonna superficie.")

# Definisco innanzitutto la regex per l'estrazione dei dati:
pattern = r'(?i)(?:mq|MQ|Mq|M2|m2|metri quadrati)\D*(\d+(?:,\d+)?(?:\.\d+)?)'

# La applico alla colonna descrizione, estraendo i valori presenti:
df['Superficie_Regex'] = df['Descrizione_Bene'].str.extract(pattern, flags = re.IGNORECASE)

# Converto la , in . per i numeri decimali:
df['Superficie_Regex'] = df['Superficie_Regex'].str.replace(',', '.')
# Converto i valori estratti in valori numerici:
df['Superficie_Regex'] = pd.to_numeric(df['Superficie_Regex'], errors='coerce')

# Mostro quanti valori sono stati estratti dalla regex applicata alla colonna Descrizione Bene:
print("La regex applicata alla colonna descrizione ha estratto N° valori:", 
      '\033[1m','\033[15;31;43m', df['Superficie_Regex'].notnull().sum(), '\033[0m')

# Mappo la colonna Superficie laddove ci sono valori nulli, con i valori estratti dalla regex:
df['Superficie'] = df['Superficie'].fillna(df['Superficie_Regex'])

# Stampo i risultati finali:
print("Ora ci sono", '\033[1m','\033[15;31;43m', df['Superficie'].isnull().sum(),
      '\033[0m', "Null Values nella colonna superficie.")
print("Sono stati quindi recuperati in totale dalla regex",
      '\033[1m','\033[15;31;43m', 35057 - 24696, '\033[0m', "valori, prima mancanti nel campo Superficie.")

# Infine elimino la colonna Superficie_Regex in quanto ormai inutile:
df = df.drop('Superficie_Regex', axis=1)

Prima di bonificare le null values, ci sono ora   35057  valori mancanti nella colonna superficie.
La regex applicata alla colonna descrizione ha estratto N° valori:   52739 
Ora ci sono   24696  Null Values nella colonna superficie.
Sono stati quindi recuperati in totale dalla regex   10361  valori, prima mancanti nel campo Superficie.


### 4 -  Mappo le Null values nel campo Numero Vani attraverso i valori estratti con la regex nel campo Superficie

- Riempio le null values con i valori della superficie dividendoli / 20, come descritto nel precedento script;
- Arrotondo i valori acquisiti, per difetto o per eccesso, a seconda del valore estratto

In [7]:
# Situazione iniziale della colonna Numero vani:
print("Numero di Null Values prima della bonifica: ", '\033[1m','\033[15;31;43m',
      df['Numero_Vani'].isnull().sum(), '\033[0m')

# Ottengo il numero di vani laddove siano assenti, dividendo la superfice del bene per 20:
df['Numero_Vani'] = df.apply(lambda row: row['Superficie'] / 20 if np.isnan(row['Numero_Vani'])
                                  else row['Numero_Vani'], axis=1)

# Funzione per arrotondare per eccesso o per difetto i float ricavati dalla divisione precedente:
def numero_vani_round(value):
    if np.isnan(value):
        return value
    decimal_part = value - int(value)
    if decimal_part >= 0.75:
        return int(value) + 1.0
    elif decimal_part >= 0.25:
        return int(value) + 0.5
    else:
        return int(value)
    
# Applico la UDF alla collonna numero vani:
df['Numero_Vani'] = df['Numero_Vani'].apply(numero_vani_round)

# Situazione finale della colonna Numero vani:
print("Numero di Null Values dopo la bonifica: ", '\033[1m','\033[15;31;43m',
      df['Numero_Vani'].isnull().sum(), '\033[0m')
print("Sono stati quindi recuperati in totale",
      '\033[1m','\033[15;31;43m', 35057 - 24696, '\033[0m', "valori, prima mancanti nel campo Numero Vani.")

Numero di Null Values prima della bonifica:    35057 
Numero di Null Values dopo la bonifica:    24696 
Sono stati quindi recuperati in totale   10361  valori, prima mancanti nel campo Numero Vani.


### 5 - Conclusioni:

- Nella colonna **Numero vani** sono stati corretti **1.341** valori, prima **uguali a 0**, ora corretti a 0.5. Un bene non può avere 0 come valore per il numero dei vani, il minimo consentito è pari a 0.5;


- Dopo aver controllato il data frame, è emerso che c'erano **27.259 valori doppi**. I valori sono quindi stati **eliminati** dal data frame in quanto rappresentavano una ridondanza di dati. Per l'eliminazione dei dati, sono state messe a confronto tutte le colonne del data frame, ed eliminati solo i dati che presentavano valori identici su tutti i campo che compongono il data frame;


- Attraverso una **regex** applicata al campo **descrizione del bene**, sono stati estratti in totale **52.739** valori inerenti alla superficie dei beni del data frame. I valori estratti sono stati quindi utilizzati per bonificare le null values che erano presenti nel campo **Superficie**, laddove non c'era alcun valore, mentre dalla regex era stato possibile estrarne uno. Infine, i valori ricavati in più rispetto a prima sono pari a **10.361**.


- Infine, avendo ottenuto dei valori in più nel campo Superficie, prima non presenti, attraverso la **proporzione** descritta nello script precedente, sono stati ricavati altrettanti valori per bonificare le null values presenti nel campo **Numero Vani**, ovvero anche in questo caso **10.361**.

### 6 - Export data frame file CSV:

In [8]:
df_csv = df.to_csv('D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\4_Dataset_Bad_Data\\Dataset_Bad_Data.csv',
                            index=False)